In [22]:
import matplotlib.pyplot as plt
import pandas as pd

#import data
price_data = pd.read_csv('Input Data/Day Data/day_data_csv.csv')
price_data.dtypes

ema(4)                     float64
high low span              float64
high above ema             float64
close above ema            float64
close relative to high     float64
volume change              float64
volume relative to span    float64
to buy                       int64
dtype: object

In [23]:
#split into x and y 
X = price_data.drop("to buy", axis=1)
y = price_data["to buy"].values.reshape(-1,1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=0)

In [37]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [38]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_test_scaled, y_test)

training_accur = dtc.score(X_train_scaled, y_train)
testing_accur = dtc.score(X_test_scaled, y_test) 

print(f"Training Accuracy Score: {training_accur}")
print(f"Testing Accuracy Score: {testing_accur}")


Training Accuracy Score: 0.49079565585766
Testing Accuracy Score: 1.0


In [27]:
importances = dtc.feature_importances_
feature_names = ['ema (4)','high low span','high above ema','close above ema','close relative to high','volume change','volume relative to span']

sorted(zip(dtc.feature_importances_, feature_names), reverse=True)

[(0.21370347322606337, 'high low span'),
 (0.15429567455136423, 'high above ema'),
 (0.15187161937414315, 'close relative to high'),
 (0.14767003379817942, 'volume relative to span'),
 (0.1410271649519515, 'volume change'),
 (0.1014556847110114, 'ema (4)'),
 (0.08997634938728682, 'close above ema')]

In [36]:
# Use our model to make predictions and generate confusion matrix
y_test_predict = dtc.predict(X_test_scaled)
y_train_predict = dtc.predict(X_train_scaled)

#price_data["Prediction"]=predict_all
#price_data.to_csv('predicted_price_data.csv')

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_test_predict))
print(confusion_matrix(y_train, y_train_predict))

[[2252    0    0    0]
 [   0 3911    0    0]
 [   0    0  283    0]
 [   0    0    0   46]]
[[1747 2596  177   36]
 [2806 4569  320   59]
 [ 184  336   58   12]
 [  29   41    9    4]]


In [39]:
#Summary of model performance
from sklearn.metrics import mean_squared_error, accuracy_score

training_score = accuracy_score(y_train, y_train_predict)
testing_score = accuracy_score(y_test, y_test_predict) 

print(f"Training Accuracy: {training_score}")
print(f"Testing Accuracy: {testing_score}")

training_mse = mean_squared_error(y_train, y_train_predict)
testing_mse = mean_squared_error(y_test, y_test_predict)

print(f"Training Mean Squared Error (MSE): {training_mse}")
print(f"Testing Mean Squared Error (MSE): {testing_mse}")

Training Accuracy: 0.49125779865978586
Testing Accuracy: 1.0
Training Mean Squared Error (MSE): 0.655318493414465
Testing Mean Squared Error (MSE): 0.0
